# Update Recent Stock Prices and Visualize
This notebook updates the `assets.db` with the latest stock prices from Alpaca, queries stock prices for a specific ticker over a date range, and plots the OHLC (Open, High, Low, Close) prices with user-selectable smoothing methods (exponential smoothing, LOWESS, or moving average).

In [ ]:
import pandas as pd
import sqlite3
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import statsmodels.api as sm
import sys
from pathlib import Path
import os

# Define BASE_DIR and adjust sys.path before importing
BASE_DIR = Path(os.getcwd()).parent  # Points to stat_656_autotrader/ from Notebooks/
sys.path.append(str(BASE_DIR))
print(f"Project root added to sys.path: {BASE_DIR}")

# Import paths from the config module
from src.config import BASE_DIR, DB_DIR

from src.utils.alpaca_utils import update_stock_prices, get_alpaca_client, fetch_alpaca_latest_bars
from src.utils.db_utils import get_db_connection, fetch_active_tickers
from src.visualizations.stock_price_trends import plot_stock_trends  # We’ll update this
from src.statistics.smoothers import smooth_lowess

ModuleNotFoundError: No module named 'config'

In [2]:
from src.utils import get_alpaca_client, fetch_alpaca_stock_tickers, fetch_alpaca_historical_data, fetch_alpaca_latest_bars

ModuleNotFoundError: No module named 'src'